<a href="https://colab.research.google.com/github/zgerem/Financial-LLM/blob/main/notebooks/Finetuning_LLM_for_Financial_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning an LLM Model for Financial News Summarizing

### Getting the News

In [1]:
!pip install transformers datasets peft bitsandbytes accelerate -q
!pip install evaluate -q
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install kaggle -q

# Upload your Kaggle API key (download from https://www.kaggle.com/account)
from google.colab import files
files.upload()

# Create Kaggle directory & move API key
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download Financial News Dataset
!kaggle datasets download -d sayelabualigah/high-quality-financial-news-dataset-for-nlp-tasks
!unzip high-quality-financial-news-dataset-for-nlp-tasks.zip -d financial_news

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/sayelabualigah/high-quality-financial-news-dataset-for-nlp-tasks
License(s): apache-2.0
  0% 0.00/1.49M [00:00<?, ?B/s]
100% 1.49M/1.49M [00:00<00:00, 162MB/s]
Archive:  high-quality-financial-news-dataset-for-nlp-tasks.zip
  inflating: financial_news/LICENSE  
  inflating: financial_news/README.md  
  inflating: financial_news/dataset.csv  


In [7]:
import pandas as pd
import json
from datasets import Dataset

# Load dataset
df = pd.read_csv("financial_news/dataset.csv")

# Keep relevant columns
df = df[["Content", "CompactedSummary"]].dropna()

# Convert into instruction-tuning format
dataset = [{"input": row["Content"], "output": row["CompactedSummary"]} for _, row in df.iterrows()]

# Convert to Hugging Face Dataset format
dataset = Dataset.from_list(dataset)

# Split dataset: 80% train, 20% eval
dataset = dataset.train_test_split(test_size=0.1, seed=42)

print(f"Training samples: {len(dataset['train'])}, Evaluation samples: {len(dataset['test'])}")
print("Train Sample:", dataset["train"][0])
print("Eval Sample:", dataset["test"][0])


✅ Training samples: 1629, Evaluation samples: 182
✅ Train Sample: {'input': "The Board of Directors of TAM Development Company invites shareholders to attend the extraordinary general assembly meeting (first meeting)  Element List Explanation\nIntroduction The Board of Directors of TAM Development Company is pleased to invite shareholders to participate and vote in the Extraordinary General Assembly meeting (first meeting), which is scheduled to be held via modern filtering methods at exactly (20:00) on Tuesday 12/05/1445 AH corresponding to 6/11/2024 AD.\nCity and Location of the General Assembly's Meeting At the company's main headquarters in Riyadh, using modern technology using the Tadawulati platform.\nHyperlink of the Meeting Location Click Here\nDate of the General Assembly's Meeting 2024-06-11 Corresponding to 1445-12-05\nTime of the General Assembly’s Meeting 20:00\nMethodology of Convening the General Assembly’s Meeting Via modern technology means\nAttendance Eligibility, Reg

### Fine-Tune the LLM Model

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/bart-large-cnn"

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
def preprocess_data(sample):
    model_inputs = tokenizer(sample["input"], max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(sample["output"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization
tokenized_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/1629 [00:00<?, ? examples/s]

Map:   0%|          | 0/182 [00:00<?, ? examples/s]

In [20]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

training_args = Seq2SeqTrainingArguments(
    output_dir="./bart-financial",
    evaluation_strategy="epoch",  # ✅ Now valid since we have an eval dataset
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,  # ✅ Enables text generation during evaluation
    fp16=True,
    report_to=None,
)

# Data collator for padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [24]:
import evaluate

# Load Rouge metric
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Decode predictions and references
    preds = [tokenizer.decode(p, skip_special_tokens=True) for p in preds]
    labels = [tokenizer.decode(l, skip_special_tokens=True) for l in labels]

    # Compute Rouge scores
    result = rouge.compute(predictions=preds, references=labels, use_stemmer=True)

    # Fix: Handle cases where Rouge returns a float instead of an object
    return {k: v["fmeasure"] if isinstance(v, dict) else v for k, v in result.items()}


In [25]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [26]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-26-935ad7046c0f>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,0.385970,0.579014,0.395978,0.504721,0.504527
2,0.101200,0.332872,0.601676,0.416660,0.525746,0.526165
3,0.123200,0.339556,0.594741,0.409611,0.518332,0.518179
4,0.080800,0.380627,0.592041,0.412282,0.522542,0.521484
5,0.053200,0.407379,0.606082,0.426509,0.532877,0.531576


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=2040, training_loss=0.08867351652360429, metrics={'train_runtime': 2256.3128, 'train_samples_per_second': 3.61, 'train_steps_per_second': 0.904, 'total_flos': 1.765106698420224e+16, 'train_loss': 0.08867351652360429, 'epoch': 5.0})

Save model to Hugging Face

In [27]:
!pip install huggingface_hub -q

In [28]:
from huggingface_hub import notebook_login

# Authenticate with Hugging Face
notebook_login()

In [29]:
# Save the trained model
trainer.save_model("./bart-financial")

# Save the tokenizer as well
tokenizer.save_pretrained("./bart-financial")

('./bart-financial/tokenizer_config.json',
 './bart-financial/special_tokens_map.json',
 './bart-financial/vocab.json',
 './bart-financial/merges.txt',
 './bart-financial/added_tokens.json',
 './bart-financial/tokenizer.json')

In [30]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("./bart-financial")
tokenizer = AutoTokenizer.from_pretrained("./bart-financial")

# Push to Hugging Face
model.push_to_hub("zgerem/bart-financial-summarizer")
tokenizer.push_to_hub("zgerem/bart-financial-summarizer")

print("✅ Model uploaded to Hugging Face Hub!")


/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

✅ Model uploaded to Hugging Face Hub!


## Evaluate the model

In [32]:
import evaluate

# Load Rouge metric
rouge = evaluate.load("rouge")

def evaluate_model(test_dataset):
    predictions, references = [], []

    for sample in test_dataset:
        input_text = sample["input"]
        reference_summary = sample["output"]

        # Generate summary using the fine-tuned model
        inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(**inputs, max_length=128, min_length=20, num_beams=4, early_stopping=True)
        generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        predictions.append(generated_summary)
        references.append(reference_summary)

    # Compute Rouge scores
    rouge_scores = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

    return rouge_scores

# Run evaluation
test_rouge_scores = evaluate_model(dataset["test"])
print("Test Set Rouge Scores:", test_rouge_scores)

✅ Test Set Rouge Scores: {'rouge1': 0.6348664457488726, 'rouge2': 0.45549100298353884, 'rougeL': 0.5669812577629332, 'rougeLsum': 0.5653480239036122}


### Manual evaluation

In [ ]:
from transformers import pipeline

# Load the summarization pipeline
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# Pick a random test news article
test_news = df_test["Content"].iloc[0]

# Generate summary
summary = summarizer(test_news, max_length=100, min_length=20, do_sample=False)
print("\n Original News:", test_news[:500])  # Print first 500 characters
print("\n Generated Summary:", summary[0]["summary_text"])
print("\n Reference Summary:", df_test["CompactedSummary"].iloc[0])